In [1]:
from codes.bq_utils import bq_query_to_dataframe
import datetime
import pandas as pd

In [3]:
limits = 'LIMIT 15000'
limits=''
sql_events = f"""
WITH geovisits_h3_indexes AS (
    SELECT *, `carto-os-eu`.h3.LONGLAT_ASH3(longitude, latitude, 10) h3_idx
    FROM `ggo-ppos-bqgis.singlespot.geovisits_paris_matview`
)
select sptId, uuid, latitude, longitude, accuracy, eventId,  
arrival, departure, score, rank, category, feature, placeName, h3_idx 
FROM geovisits_h3_indexes {limits}
"""

In [4]:
df = bq_query_to_dataframe(sql_events)
df.head()

,sptId,uuid,latitude,longitude,accuracy,eventId,arrival,departure,score,rank,category,feature,placeName,h3_idx
0,096cbba6-de4c-4a2b-aeab-8f74d576063c,46524783-a8d1-4255-814e-935f5e27753a,48.858984,2.347201,39.0,f0fb50ec11ad194ec48e325b9c652fceb85bc1783f2d0f...,2021-07-13 11:31:46 +0200,2021-07-13 11:42:31 +0200,0.995102,1,shop,beauty,Yves Rocher,8a1fb4662c8ffff
1,6dd9981e-4c47-4991-8413-21b7606e73dd,55f6e501-5fec-4d31-adaa-42395440593e,48.859652,2.352528,57.0,09fdf6a8a52a0b6a438b029bcb37e203fb2436080b8c77...,2021-07-15 10:26:42 +0200,2021-07-15 10:44:27 +0200,0.499995,1,amenity,arts_centre,Centre Georges Pompidou,8a1fb4662ccffff
2,2f068bfa-32b1-436e-9f87-c98c2e5e0352,a8a7353d-2c69-4855-8cb2-37ae74bb1c81,48.844031,2.405675,129.0,f-e3b7e868fa9184194561cd24a4af84a7f0800336,2021-07-15 08:12:27 +0200,2021-07-15 08:23:36 +0200,0.243552,1,shop,florist,None,8a1fb4645037fff
3,2f068bfa-32b1-436e-9f87-c98c2e5e0352,a8a7353d-2c69-4855-8cb2-37ae74bb1c81,48.845534,2.403654,45.0,f-210b3e3123bbc80ea7acf8d71a3141e48140c94d,2021-07-15 08:36:51 +0200,2021-07-15 08:46:34 +0200,0.368871,1,shop,greengrocer,Tony Fruits,8a1fb4645117fff
4,2f068bfa-32b1-436e-9f87-c98c2e5e0352,a8a7353d-2c69-4855-8cb2-37ae74bb1c81,48.844027,2.405677,264.0,f-793feea4df2c9b887ceb1f30ff09754116b95c4a,2021-07-15 06:07:21 +0200,2021-07-15 06:25:55 +0200,0.137389,1,leisure,dance,None,8a1fb4645037fff


In [5]:
df['sptId'] = df['sptId'].astype(str)
df['category'] = df['category'].astype(str)
df['feature'] = df['feature'].astype(str)
df['arrival'] = pd.to_datetime(df['arrival'], format='%Y-%m-%d %H:%M:%S', utc=True)
df['depature'] = pd.to_datetime(df['departure'], format='%Y-%m-%d %H:%M:%S', utc=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481634 entries, 0 to 481633
Data columns (total 15 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   sptId      481634 non-null  object             
 1   uuid       481634 non-null  object             
 2   latitude   481634 non-null  float64            
 3   longitude  481634 non-null  float64            
 4   accuracy   481634 non-null  float64            
 5   eventId    481634 non-null  object             
 6   arrival    481634 non-null  datetime64[ns, UTC]
 7   departure  481634 non-null  object             
 8   score      481634 non-null  float64            
 9   rank       481634 non-null  int64              
 10  category   481634 non-null  object             
 11  feature    481634 non-null  object             
 12  placeName  325184 non-null  object             
 13  h3_idx     481634 non-null  object             
 14  depature   481634 non-null  datetime

In [6]:
df_w = pd.DataFrame(df[['h3_idx', 'category', 'feature']])
df_w['count'] = 1
df_w.head()


,h3_idx,category,feature,count
0,8a1fb4662c8ffff,shop,beauty,1
1,8a1fb4662ccffff,amenity,arts_centre,1
2,8a1fb4645037fff,shop,florist,1
3,8a1fb4645117fff,shop,greengrocer,1
4,8a1fb4645037fff,leisure,dance,1


In [7]:
grouped_columns = ['category', 'feature']
grouped_columns = ['category']

h3_pivot = pd.DataFrame(pd.pivot_table(df_w,  columns=grouped_columns, index='h3_idx', values="count", aggfunc=[len], fill_value=0, margins = True, margins_name='total')).reset_index()
h3_pivot.columns = h3_pivot.columns.map('_'.join).str.replace('len_', '')

h3_pivot.rename(columns={ h3_pivot.columns[0]: "h3_idx" }, inplace=True)
h3_pivot.head()

,h3_idx,aeroway,amenity,building,club,craft,highway,historic,landuse,leisure,man_made,natural,office,railway,shop,tourism,user,total
0,8a1fb460d827fff,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,8,10
1,8a1fb460d82ffff,0,2,0,0,0,0,0,0,0,0,0,0,0,6,0,0,8
2,8a1fb460d907fff,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2
3,8a1fb460d90ffff,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,4
4,8a1fb460d927fff,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,6


In [8]:
# 
indexNames = h3_pivot[h3_pivot['h3_idx'] == 'total'].index
h3_pivot.drop(indexNames, inplace=True)
# h3_pivot[h3_pivot['h3_idx'] == 'total']
# h3_pivot.info()


In [9]:
h3_pivot.head()

,h3_idx,aeroway,amenity,building,club,craft,highway,historic,landuse,leisure,man_made,natural,office,railway,shop,tourism,user,total
0,8a1fb460d827fff,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,8,10
1,8a1fb460d82ffff,0,2,0,0,0,0,0,0,0,0,0,0,0,6,0,0,8
2,8a1fb460d907fff,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2
3,8a1fb460d90ffff,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,4
4,8a1fb460d927fff,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,6


In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [10]:
df_names = h3_pivot.columns
df_names = df_names[1:]
df_names

Index(['aeroway', 'amenity', 'building', 'club', 'craft', 'highway',
       'historic', 'landuse', 'leisure', 'man_made', 'natural', 'office',
       'railway', 'shop', 'tourism', 'user', 'total'],
      dtype='object')

In [11]:
import seaborn as sns
h3_pivot_corr=h3_pivot[df_names]
correlations= h3_pivot_corr.corr()
mask = np.zeros_like(correlations)  # make mask
mask[np.triu_indices_from(mask)] = True  # mask the upper triangle
fig, ax = plt.subplots(figsize=(11, 9))  # create a figure and a subplot
cmap = sns.diverging_palette(220, 10, as_cmap=True)  # custom color map
"""
sns.heatmap(
    correlations,
    mask=mask,
    cmap=cmap,
    center=0,
    linewidth=0.5,
    cbar_kws={'shrink': 0.5}
)
"""
sns.heatmap(
    correlations,
    annot=True
)

NameError: name 'np' is not defined

In [ ]:
correlations['total']

In [ ]:
import tensorflow as tf

In [ ]:
X = h3_pivot_corr.drop('total',axis=1)
y = h3_pivot_corr['total']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([
    ('std_scalar', StandardScaler())
])
X_train= pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

# GOOGLE TF Model

In [ ]:
import codes.tf_model_1 as tf1

In [ ]:
learning_rate = 0.05
epochs = 400
batch_size = 30

# Specify the feature and the label.
my_feature = "shop"  # the total number of rooms on a specific city block.
my_label="total" # the median value of a house on a specific city block.
# That is, you're going to create a model that predicts house value based 
# solely on total_rooms.  

# Discard any pre-existing version of the model.
my_model = None

In [ ]:
# Invoke the functions.
my_model = tf1.build_model(learning_rate)

In [ ]:
pd_X_train = pd.DataFrame(X_train, columns=X.columns)
pd_y_train = pd.DataFrame(y_train, columns=['total'])

In [ ]:
weight, bias, epochs, rmse = tf1.train_model(my_model, pd_X_train[my_feature], pd_y_train[my_label], epochs=epochs, batch_size=batch_size)

In [ ]:
print("\nThe learned weight for your model is %.4f" % weight)
print("The learned bias for your model is %.4f\n" % bias )

#tf1.plot_the_model(trained_weight=weight, trained_bias=bias, feature=my_feature, label=my_label)
tf1.plot_the_loss_curve(epochs, rmse)

# Training Linear Regression model

In [ ]:
import hvplot.pandas

from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression(normalize=True)
lin_reg.fit(X_train,y_train)

In [ ]:
# print the intercept
print(lin_reg.intercept_)

In [ ]:
pred = lin_reg.predict(X_test)

In [ ]:
pd.DataFrame({'True Values': y_test, 'Predicted Values': pred}).hvplot.scatter(x='True Values', y='Predicted Values')

In [ ]:
pd.DataFrame({'Error Values': (y_test - pred)}).hvplot.kde()

In [ ]:
test_pred = lin_reg.predict(X_test)
train_pred = lin_reg.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df = pd.DataFrame(data=[["Linear Regression", *evaluate(y_test, test_pred) , cross_val(LinearRegression())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])

## Rodust Regression

### Random Sample Consensus - RANSAC

In [ ]:
from sklearn.linear_model import RANSACRegressor

model = RANSACRegressor(base_estimator=LinearRegression(), max_trials=100)
model.fit(X_train, y_train)

test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Robust Regression", *evaluate(y_test, test_pred) , cross_val(RANSACRegressor())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

## Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=100, solver='cholesky', tol=0.0001, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)

test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Ridge Regression", *evaluate(y_test, test_pred) , cross_val(Ridge())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

## LASSO Regression

In [ ]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.1, 
              precompute=True, 
#               warm_start=True, 
              positive=True, 
              selection='random',
              random_state=42)
model.fit(X_train, y_train)

test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Lasso Regression", *evaluate(y_test, test_pred) , cross_val(Lasso())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

## Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet

model = ElasticNet(alpha=0.1, l1_ratio=0.9, selection='random', random_state=42)
model.fit(X_train, y_train)

test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Elastic Net Regression", *evaluate(y_test, test_pred) , cross_val(ElasticNet())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

## Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_reg = PolynomialFeatures(degree=2)

X_train_2_d = poly_reg.fit_transform(X_train)
X_test_2_d = poly_reg.transform(X_test)

lin_reg = LinearRegression(normalize=True)
lin_reg.fit(X_train_2_d,y_train)

test_pred = lin_reg.predict(X_test_2_d)
train_pred = lin_reg.predict(X_train_2_d)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Polynomail Regression", *evaluate(y_test, test_pred), 0]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True)

## Stochastic Gradient Descent

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(n_iter_no_change=250, penalty=None, eta0=0.0001, max_iter=100000)
sgd_reg.fit(X_train, y_train)

test_pred = sgd_reg.predict(X_test)
train_pred = sgd_reg.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Stochastic Gradient Descent", *evaluate(y_test, test_pred), 0]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True)

## Artficial Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

model = Sequential()

model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.2))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))

model.compile(optimizer=Adam(0.00001), loss='mse')

r = model.fit(X_train, y_train,
              validation_data=(X_test,y_test),
              batch_size=1,
              epochs=100)

In [ ]:
pd.DataFrame({'True Values': y_test, 'Predicted Values': pred}).hvplot.scatter(x='True Values', y='Predicted Values')

In [ ]:
pd.DataFrame(r.history).hvplot.line(y=['loss', 'val_loss'])

In [ ]:
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Artficial Neural Network", *evaluate(y_test, test_pred), 0]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True)

## Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=1000)
rf_reg.fit(X_train, y_train)

test_pred = rf_reg.predict(X_test)
train_pred = rf_reg.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Random Forest Regressor", *evaluate(y_test, test_pred), 0]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True)

## Support Vector Machine

In [ ]:
""" from sklearn.svm import SVR

svm_reg = SVR(kernel='rbf', C=1000000, epsilon=0.001)
svm_reg.fit(X_train, y_train)

test_pred = svm_reg.predict(X_test)
train_pred = svm_reg.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["SVM Regressor", *evaluate(y_test, test_pred), 0]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True) """

# Results

In [ ]:
results_df

## Models Comparison

In [ ]:
results_df.set_index('Model', inplace=True)
results_df['R2 Square'].plot(kind='barh', figsize=(12, 8))